In [1]:
# Read CSV file from blob storage
storage_account_name = 'str1wk6'
storage_account_access_key = 'OnTwDpJa14kT8Xi9rY36r9kKeUqNXJdCm1D4RLl9LgUUGBFdmKjQoWhpn2dljkfDAhuDXEprXyB7Ld0eFW+upQ=='
containerName = 'cnt1'
fileName = 'test.csv'
# Get data of test csv file from blob storage
spark.conf.set("fs.azure.account.key."+storage_account_name+".blob.core.windows.net",storage_account_access_key)
df = spark.read.format('csv').options(header='true',inferSchema='true').load('wasbs://cnt1@str1wk6.blob.core.windows.net/Test.csv')

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
x = df.select('Age','EstimatedSalary','Gender').toPandas()
y = df.select('Purchased').toPandas()
Gender = pd.get_dummies(x['Gender'])
x = x.drop('Gender', axis=1)
x = pd.concat([x, Gender], axis=1)
scX = StandardScaler()
x = scX.fit_transform(x)
print(x)

[[ -1.22160328e+00 -1.16137246e+00 -7.63762616e-01 7.63762616e-01]
 [ 6.35233707e-01 -1.13076763e+00 -7.63762616e-01 7.63762616e-01]
 [ -4.09237100e-01 -4.26856728e-01 1.30930734e+00 -1.30930734e+00]
 [ -2.93184788e-01 1.61078010e-03 1.30930734e+00 -1.30930734e+00]
 [ -1.22160328e+00 5.83102398e-01 -7.63762616e-01 7.63762616e-01]
 [ -2.93184788e-01 3.22156021e-02 -7.63762616e-01 7.63762616e-01]
 [ -2.93184788e-01 8.27940974e-01 1.30930734e+00 -1.30930734e+00]
 [ 2.87076771e-01 2.84785923e+00 1.30930734e+00 -1.30930734e+00]
 [ -5.25289411e-01 -7.32904948e-01 -7.63762616e-01 7.63762616e-01]
 [ 6.35233707e-01 2.46449356e-01 1.30930734e+00 -1.30930734e+00]
 [ -4.09237100e-01 7.05521686e-01 1.30930734e+00 -1.30930734e+00]
 [ -4.09237100e-01 -1.51413330e-01 1.30930734e+00 -1.30930734e+00]
 [ -1.10555097e+00 8.89150618e-01 -7.63762616e-01 7.63762616e-01]
 [ 2.87076771e-01 -1.19197728e+00 -7.63762616e-01 7.63762616e-01]
 [ -1.33765559e+00 7.66731330e-01 -7.63762616e-01 7.63762616e-01]
 [ -6.10801641e-02 7.05521686e-01 -7.63762616e-01 7.63762616e-01]
 [ 2.02786145e+00 -9.77743524e-01 -7.63762616e-01 7.63762616e-01]
 [ 1.79575683e+00 -9.47138702e-01 -7.63762616e-01 7.63762616e-01]
 [ 1.91180914e+00 -8.85929058e-01 -7.63762616e-01 7.63762616e-01]]

In [3]:
# Download file from blob storage
from azure.storage.blob import *
import pickle
import os

if not os.path.exists('/opt/pkl_folder'):
  os.mkdir('/opt/pkl_folder')

blob_storage = BlockBlobService(account_name=storage_account_name, account_key=storage_account_access_key)
blob_storage.get_blob_to_path(containerName, 'DecisionTreeClassification.pkl', '/opt/pkl_folder/DecisionTreeClassification.pkl')

pkl_file = open('/opt/pkl_folder/DecisionTreeClassification.pkl', 'rb')
classifier = pickle.load(pkl_file)

In [4]:
from sklearn.metrics import r2_score
y_pred = classifier.predict(x)
accuracy = r2_score(y, y_pred)
print("Accuracy", accuracy)

Accuracy -0.583333333333

In [5]:
y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])
x_df = pd.DataFrame(x, columns=['x1','x2','x3','x4'])
result_df = pd.concat([x_df, y_pred_df], axis=1)
spark_result_df = spark.createDataFrame(result_df)1:07:3
spark_result_df.show()

+--------------------+--------------------+-------------------+-------------------+------+
 x1| x2| x3| x4|y_pred|
+--------------------+--------------------+-------------------+-------------------+------+
 -1.2216032825403857| -1.1613724556702252|-0.7637626158259733| 0.7637626158259734| 0|
 0.6352337069210005| -1.1307676336761416|-0.7637626158259733| 0.7637626158259734| 1|
 -0.4092370996510293|-0.42685672781221873| 1.3093073414159542|-1.3093073414159542| 0|
-0.29318478780969265|0.001610780104951...| 1.3093073414159542|-1.3093073414159542| 0|
 -1.2216032825403857| 0.5831023979925402|-0.7637626158259733| 0.7637626158259734| 0|
-0.29318478780969265| 0.03221560209903531|-0.7637626158259733| 0.7637626158259734| 0|
-0.29318478780969265| 0.8279409739452089| 1.3093073414159542|-1.3093073414159542| 1|
 0.28707677139699056| 2.8478592255547266| 1.3093073414159542|-1.3093073414159542| 1|
 -0.525289411492366| -0.7329049477530547|-0.7637626158259733| 0.7637626158259734| 0|
 0.6352337069210005| 0.24644935605762053| 1.3093073414159542|-1.3093073414159542| 1|
 -0.4092370996510293| 0.7055216859688745| 1.3093073414159542|-1.3093073414159542| 1|
 -0.4092370996510293| -0.1514133298654663| 1.3093073414159542|-1.3093073414159542| 0|
 -1.1055509706990492| 0.8891506179333761|-0.7637626158259733| 0.7637626158259734| 0|
 0.28707677139699056| -1.1919772776643087|-0.7637626158259733| 0.7637626158259734| 0|
 -1.3376555943817223| 0.7667313299570417|-0.7637626158259733| 0.7637626158259734| 0|
-0.06108016412701...| 0.7055216859688745|-0.7637626158259733| 0.7637626158259734| 1|
 2.02786144901704| -0.9777435237057236|-0.7637626158259733| 0.7637626158259734| 1|
 1.795756825334367| -0.9471387017116399|-0.7637626158259733| 0.7637626158259734| 1|
 1.9118091371757036| -0.8859290577234727|-0.7637626158259733| 0.7637626158259734| 1|
+--------------------+--------------------+-------------------+-------------------+------+

In [6]:
# Save dataframe to cosmosDB
writeConfig = {
 "Endpoint" : "https://predictresultdb.documents.azure.com:443/",
 "Masterkey" : "RC9QHfB7DAbpKkXT8spsuYNnYFBW07mYlpYuZATc4lk9atAWTKpoz3Gzzhe3M6cbHhFjEmppEExqDeBmP8EeVA==",
 "Database" : "ToDoList",
 "Collection" : "week6Assignment",
 "Upsert" : "true"
}

spark_result_df.write.format("com.microsoft.azure.cosmosdb.spark").mode('overwrite').options(**writeConfig).save()